**Zadanie 4. (6p)** Napisz program, korygujący błędy, który czyta ze standardowego wejścia plik
zawierający wiersze, w których mamy pary: (słowo-poprawne, słowo-wpisane). Przeczytawszy parę,
dokonuje korekty i sprawdza, czy jest taka jak trzeba. Wypisuje błędy i podlicza na koniec ich
procentowy udział we wszystkich korektach. Oczywiście korzysta z informacji o poprawnym słowie
**tylko** do sprawdzenia, czy popełnił błąd. Dla zbioru testowego 1 powinien mieć skuteczność ponad
70%. Wymagana skuteczność dla zbioru testowego 2 zostanie podana później.

Uwaga: nie wolno spamiętywać żadnych danych z poprzednich korekt (dane są tak skontruowane,
że taka strategia dawałaby „nieuczciwą” przewagę programowi z niej korzystającemu). Uwaga: na
przyszłych zajęciach zrobimy osobno punktowany konkurs poprawiania literówek.

In [1]:
import csv
import string
import logging
from heapq import heappush, heappop

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
TEST_DATASET_FILE_PATH = 'data/literowki1.txt'
BASE_FORMS_FILE_PATH = 'data/polimorfologik-2.1.txt'

In [4]:
with open(BASE_FORMS_FILE_PATH) as f:
    _ALL_WORDS = {word for _base, word, *_ in csv.reader(f, delimiter=';')}
    
def is_correct(word):
    return word in _ALL_WORDS

In [5]:
POLISH_CHARACTERS = 'ąćęłńóśżź'
ASCII_BY_POLISH = {pol: asc for pol, asc in zip(POLISH_CHARACTERS, 'acelnoszz')}
ALL_CHARACTERS = string.ascii_lowercase + POLISH_CHARACTERS

In [14]:
def all_deletions(word):
    for i in range(len(word)):
        yield f'{word[:i]}{word[i + 1 :]}'

def all_insertions(word):
    for i in range(len(word)):
        for char in ALL_CHARACTERS:
            yield f'{word[:i]}{char}{word[i:]}'

def all_substitutions(word):
    for i in range(len(word)):
        for char in ALL_CHARACTERS:
            yield f'{word[:i]}{char}{word[i + 1 :]}'

def all_polish_substitutions(word):
    """Polish to ASCII and vice versa"""
    for i, orig in enumerate(word):
        char = None
        if pol := ASCII_BY_POLISH.get(orig):
            char = pol
        else:
            for k, v in ASCII_BY_POLISH.items():
                if v == orig:
                    char = k
        if char:
            yield f'{word[:i]}{char}{word[i + 1 :]}'
            
def all_ascii_substitutions(word):
    for i in range(len(word)):
        for char in string.ascii_lowercase:
            yield f'{word[:i]}{char}{word[i + 1 :]}'

def all_swaps(word):
    for i in range(len(word) - 1):
        for j in range(i + 1, len(word)):
            yield f'{word[:i]}{word[j]}{word[i + 1:j]}{word[i]}{word[j + 1 :]}'

EDITS_COSTS = {
    all_deletions: 2,
    all_insertions: 2,
    #all_substitutions: 1,
    all_polish_substitutions: 1,
    all_ascii_substitutions: 2,
    all_swaps: 1,
}            

def all_edits(word):
    """Yields words within one edit operation of the given `word`
    and their respective costs.
    """
    for gen, cost in EDITS_COSTS.items():
        for edit in gen(word):
            yield cost, edit

In [15]:
def stream_edits(word, max_cost=3):
    q = []
    cost = 0
    heappush(q, (cost, word))
    while cost < max_cost:
        cost, word = heappop(q)
        for additional_cost, edit in all_edits(word):
            yield edit
            heappush(q, (cost + additional_cost, edit))

In [18]:
def is_likely(word, edit):
    return is_correct(edit) and word[0] == edit[0] and word[-1] == edit[-1]

In [21]:
def correct(word):
    for edit in stream_edits(word):
        if is_likely(word, edit):
            return edit

In [22]:
with open(TEST_DATASET_FILE_PATH) as f:
    num_samples = 0
    correct_results = 0
    for line in f:
        num_samples += 1
        expected, word = line.split()
        actual = correct(word)
        if expected == actual:
            correct_results += 1
        logging.info('%s %s %s', word, expected, actual)
    print(correct_results / num_samples)

INFO:root:lokomowtuwa lokomotywa lokomotywa
INFO:root:kolokotywa lokomotywa kolorytowa
INFO:root:lokonowaywa lokomotywa None
INFO:root:kolomotywa lokomotywa kolorytowa
INFO:root:lokotaywa lokomotywa None
INFO:root:prawodpodoniestso prawdopodobieństwo None
INFO:root:prawdopopdobieństwo prawdopodobieństwo prawdopodobieństwo
INFO:root:prawdopodobieńśtwo prawdopodobieństwo prawdopodobieństwo
INFO:root:prawodpodoniesnitswo prawdopodobieństwo None
INFO:root:prawwdopodobeinistwo prawdopodobieństwo None
INFO:root:prawdopodobiensitwao prawdopodobieństwo None
INFO:root:prawdopodonbieńśtwo prawdopodobieństwo prawdopodobieństwo
INFO:root:prawdopdoodobienśtwo prawdopodobieństwo None
INFO:root:prawdopodbieństwi prawdopodobieństwo None
INFO:root:prawodpoodbieństwo prawdopodobieństwo prawdopodobieństwo
INFO:root:prawdomównu prawdomówny prawdomównemu
INFO:root:proawdomówny prawdomówny prawdomówny
INFO:root:prawdowny prawdomówny prawdzony
INFO:root:kompteurerk komputerek komputerek
INFO:root:komputerer 

0.5289256198347108
